In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from tqdm import tqdm

In [3]:
EXP_NAME = "exp120"
DIR = "/gdrive/MyDrive/amazon_kdd_2023/"
WEIGHTS = [1.0, 1.0, 1.0, 1.0]

In [4]:
subs = {
    "exp115": DIR + "data/output/task1/exp115.parquet",
    "exp116": DIR + "data/output/task1/exp116.parquet",
    "exp117": DIR + "data/output/task1/exp117.parquet",
    "exp123": DIR + "data/output/task1/exp123.parquet",
}

In [5]:
assert(len(WEIGHTS) == len(subs))

In [6]:
for i, n in enumerate(subs):
    path = subs[n]
    if i == 0:
        sub = pd.read_parquet(path)
        sub.columns = ['locale', 'labels0']
    else:
        sub[f'labels{i}'] = pd.read_parquet(path)['next_item_prediction']

In [7]:
def cust_blend(dt, W = [1,1,1,1], base=3):   
    REC = []
    for i in range(len(W)):
        REC.append(dt[f'labels{i}'])

    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+base))
            else:
                res[v] = (W[M]/(n+base))
    
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())
    
    return res[:100]

In [8]:
sub['next_item_prediction'] = sub.apply(cust_blend, W = WEIGHTS, base=4, axis=1)

In [9]:
sub[["locale", "next_item_prediction"]].to_parquet(DIR + "data/output/task1/" + EXP_NAME + ".parquet", engine='pyarrow')

In [10]:
sub.head()

,locale,labels0,labels1,labels2,labels3,next_item_prediction
0,DE,"[B07SDFLVKD, B091CK241X, B0BGC82WVW, B093X59B3...","[B07SDFLVKD, B091CK241X, B0BGC82WVW, B093X59B3...","[B07SDFLVKD, B091CK241X, B0BGC82WVW, B093X59B3...","[B07SDFLVKD, B091CK241X, B0BGC82WVW, B093X59B3...","[B07SDFLVKD, B091CK241X, B0BGC82WVW, B093X59B3..."
1,DE,"[B084CB7GX9, B004P4QFJM, B08HQWQ1SK, B08XW6P9P...","[B084CB7GX9, B08HQWQ1SK, B004P4QFJM, B08XW6P9P...","[B084CB7GX9, B0B7KCP6SL, B004P4QFJM, B004P4OF1...","[B084CB7GX9, B08HQWQ1SK, B004P4QFJM, B004P4OF1...","[B084CB7GX9, B004P4QFJM, B08HQWQ1SK, B004P4OF1..."
2,DE,"[B09Z4PZQBF, B07HFTJLR8, B09KBCTXF5, B09GPJ15G...","[B09Z4PZQBF, B07HFTJLR8, B09KBCTXF5, B09GPJ15G...","[B09Z4PZQBF, B09KBCTXF5, B07HFTJLR8, B08LLF9M1...","[B09Z4PZQBF, B09KBCTXF5, B07HFTJLR8, B08LLF9M1...","[B09Z4PZQBF, B09KBCTXF5, B07HFTJLR8, B09GPJ15G..."
3,DE,"[B07Y1KLF25, B07T5XY2CJ, B07SZHKGZJ, B07VSNX4G...","[B07Y1KLF25, B07T5XY2CJ, B07SZHKGZJ, B07YCFFC4...","[B07Y1KLF25, B07T5XY2CJ, B07QQZD49D, B07VSNX4G...","[B07Y1KLF25, B07T5XY2CJ, B07SZHKGZJ, B07QQZD49...","[B07Y1KLF25, B07T5XY2CJ, B07SZHKGZJ, B07QQZD49..."
4,DE,"[B0B2JY9THB, B08SXLWXH9, B08P94RML3, B08YK8FQJ...","[B0B2JY9THB, B08SXLWXH9, B08P94RML3, B08YK8FQJ...","[B0B2JY9THB, B08SXLWXH9, B08P94RML3, B08YK8FQJ...","[B0B2JY9THB, B08SXLWXH9, B08P94RML3, B08YK8FQJ...","[B0B2JY9THB, B08SXLWXH9, B08P94RML3, B08YK8FQJ..."


In [11]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316972 entries, 0 to 316971
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   locale                316972 non-null  object
 1   labels0               316972 non-null  object
 2   labels1               316972 non-null  object
 3   labels2               316972 non-null  object
 4   labels3               316972 non-null  object
 5   next_item_prediction  316972 non-null  object
dtypes: object(6)
memory usage: 14.5+ MB


In [12]:
len(sub)

316972